In [3]:
import os
import pandas as pd
import numpy as np
import re
from pathlib import Path  

import yfinance as yf
import plotly.express as px
import plotly.graph_objects as go
from pathlib import Path

from sklearn.preprocessing import StandardScaler 

In [5]:
DATA_KMEANS = Path("data/kmeans")
OUTPUT_DIR = Path("output/dashboard")

for p in (DATA_KMEANS, OUTPUT_DIR):
    p.mkdir(parents=True, exist_ok=True)

In [20]:

df = pd.read_csv(os.path.join(DATA_KMEANS, "kmeans_watchlist.csv"))

interesting_signals = ["STRONG BUY", "BUY"]
df_filtered = df[df["final_signal"].isin(interesting_signals)]
df_filtered.head()

,symbol,HotScore,MomentumScore,VolumeScore,VolumeSpike,VolatilityScore,TrendScore,cluster,watch_label,final_signal
4,ARWR,0.956380,1.000000,0.976562,2.501594,0.955729,0.734375,1,🔥 HOT,STRONG BUY
41,SYM,0.957292,0.989583,0.973958,2.340527,0.940104,0.820312,1,🔥 HOT,STRONG BUY
46,URBN,0.958464,0.986979,0.994792,4.050246,0.921875,0.804688,1,🔥 HOT,STRONG BUY
54,ARWR,0.956380,1.000000,0.976562,2.501594,0.955729,0.734375,1,🔥 HOT,STRONG BUY
91,SYM,0.957292,0.989583,0.973958,2.340527,0.940104,0.820312,1,🔥 HOT,STRONG BUY


In [21]:
df_unique = df_filtered.drop_duplicates(subset="symbol")
df_unique.shape


(84, 10)

In [22]:
top50 = df_unique.sort_values("HotScore", ascending=False).head(50)
symbols_to_fetch = top50["symbol"].tolist()
 

In [24]:
# ===============================
# 4️⃣ Robust yfinance Fetch Function
# ===============================
def fetch_yf_robust(symbols):
    data = []
    for s in symbols:
        t = yf.Ticker(s)
        try:
            # Try fast_info
            fi = t.fast_info
            current_price = fi.get("last_price")
            daily_change_pct = fi.get("last_change_pct")
            market_cap = fi.get("market_cap")
            volume = fi.get("last_volume")
            
            # Fallback to info
            if current_price is None:
                info = t.info
                current_price = info.get("regularMarketPrice")
                daily_change_pct = info.get("regularMarketChangePercent")
                market_cap = info.get("marketCap")
                volume = info.get("regularMarketVolume")
            
            # Fallback to history
            if current_price is None:
                hist = t.history(period="2d")
                if not hist.empty and len(hist) >= 2:
                    current_price = hist["Close"].iloc[-1]
                    daily_change_pct = (hist["Close"].iloc[-1] - hist["Close"].iloc[-2]) / hist["Close"].iloc[-2] * 100
                    volume = hist["Volume"].iloc[-1]
            
            data.append({
                "symbol": s,
                "currentPrice": current_price,
                "dailyChangePct": daily_change_pct,
                "marketCap": market_cap,
                "volume": volume
            })
        except Exception as e:
            print(f"Error fetching {s}: {e}")
            data.append({"symbol": s, "currentPrice": None, "dailyChangePct": None, "marketCap": None, "volume": None})
    return pd.DataFrame(data)

# Fetch data
yf_data = fetch_yf_robust(symbols_to_fetch)
df_top50 = top50.merge(yf_data, on="symbol", how="left")
df_top50.head()


,symbol,HotScore,MomentumScore,VolumeScore,VolumeSpike,VolatilityScore,TrendScore,cluster,watch_label,final_signal,currentPrice,dailyChangePct,marketCap,volume
0,AXSM,1.000000,1.000000,1.000000,1.369138,1.000000,1.000000,1,🔥 HOT,STRONG BUY,177.27,1.797407,8936648704,618951
1,UNF,0.989695,0.992366,0.984733,1.163488,0.992366,0.992366,1,🔥 HOT,STRONG BUY,209.80,-1.228760,3793564672,132996
2,RHLD,0.979844,0.981250,0.965625,1.705708,0.993750,0.996875,1,🔥 HOT,STRONG BUY,206.34,-8.848340,1759254656,112321
3,RVMD,0.978063,0.997151,0.980057,4.115478,0.991453,0.877493,1,🔥 HOT,STRONG BUY,120.28,-2.425568,23252506624,4798696
4,ULTA,0.976692,0.962406,0.977444,1.956608,0.992481,0.992481,1,🔥 HOT,STRONG BUY,663.48,-0.246574,29749356544,391923


In [93]:
import plotly.graph_objects as go
import os

# ===============================
# Define easy-to-change colors for each bar
# ===============================
BAR_COLORS = [
    "#32CD32",  # HotScore → lime green
    "#228B22",  # MomentumScore → dark green
    "#FF4136",  # VolumeScore → red
    "#FFA500",  # Volume Spike → orange
    "#808080",  # VolatilityScore → gray
    "#000000"   # TrendScore → black
]

# ===============================
# Dashboard function
# ===============================
def create_stock_dashboard(df_stock, output_path=None):
    row = df_stock.iloc[0]

    # Names and values for 6 bars
    metrics = ["HotScore","MomentumScore","VolumeScore","VolumeSpike","VolatilityScore","TrendScore"]
    values = [row[m] for m in metrics]

    # Assign colors (one per bar)
    colors = BAR_COLORS

    # Create figure
    fig = go.Figure()

    fig.add_trace(go.Bar(
        x=metrics,
        y=values,
        marker=dict(color=colors, line=dict(color='black', width=1)),
        text=[f"{v:.3f}" for v in values],
        textposition='outside',
        name='Metrics'
    ))

    # Layout
    title_text = (
        f"{row.symbol} – {row.final_signal} | {row.watch_label} | "
        f"Price: ${row.currentPrice:.2f} ({row.dailyChangePct:.2f}%) | "
        f"MarketCap: {row.marketCap:,} | Volume: {row.volume:,}"
    )

    fig.update_layout(
        template=None,
        paper_bgcolor='black',
        plot_bgcolor='black',
        height=500,
        title=dict(text=title_text, font=dict(color='white', size=16)),
        yaxis=dict(title='Value', tickfont=dict(color='white')),
        xaxis=dict(tickfont=dict(color='white')),
        barmode='group'
    )

    # Save HTML if needed
    if output_path:
        fig.write_html(output_path, include_plotlyjs='cdn')
        

In [92]:
 
for stock in df_top50.itertuples():
    symbol = stock.symbol
    output_file = os.path.join(OUTPUT_DIR, f"{symbol}.html")
    
    # Select row as DataFrame
    df_row = df_top50[df_top50['symbol']==symbol]
    
    create_stock_dashboard(df_row, output_path=output_file)

